## Demonstrating OpenQASM3
The preferred path to send a dynamic circuit to IBM Quantum hardware is through `backend.run(program, dynamic=True)`. However, currently Qiskit cannot describe all of the capabilities IBM backend's are capable of such as classical compute, and `extern` function invocations. Similarily, not everyone is using Qiskit and Python, and they may require another way to send programs to the backend.

We are hard at work adding more capability support to Qiskit, however, in the meantime we support accepting OpenQASM 3 programs directly through the `qasm3-runner` Qiskit Runtime program.

Below we import the Qiskit Runtime provider.

In [1]:
from qiskit import transpile
from qiskit_ibm_runtime import QiskitRuntimeService

import logging.config
logging.config.dictConfig({
    'version': 1,
    'disable_existing_loggers': True,
})


In [2]:
hub = "ibm-q-internal"
group = "performance"
project = "summit-demos"
hgp = f"{hub}/{group}/{project}"
backend_name = "ibm_peekskill"

# Temporary overrides.
hgp = "ibm-q-internal/dev-sys-software/internal-test"
backend_name = "ibm_peekskill"

service = QiskitRuntimeService(instance=hgp)
runtime_backend = service.backend(backend_name, instance=hgp)

## We can dump circuits from Qiskit to OpenQASM 3
The exporter has several configuration settings, however, those below are typically valid for IBM Quantum backends.

When debugging your program it is very useful to be able to dump a circuit to a QASM source string.

In [3]:
from qiskit import QuantumCircuit
qc_reset = QuantumCircuit(1, 2)
qc_reset.x(0)
qc_reset.measure(0, 0)
qc_reset.x(0).c_if(0, 1)
qc_reset.measure(0, 1)
qc_reset = transpile(qc_reset, runtime_backend)

In [4]:
from qiskit import qasm3

exporter = qasm3.Exporter(disable_constants=True, basis_gates=runtime_backend.configuration().basis_gates)

print(qasm3_reset:=exporter.dumps(qc_reset))

OPENQASM 3;
include "stdgates.inc";
bit[2] c;
x $0;
c[0] = measure $0;
if (c[0] == 1) {
  x $0;
}
c[1] = measure $0;



## Submitting the OpenQASM 3 source string directly to the backend
There are some limitations to submitting source strings directly (such as using the exporter settings above). Please see the dynamic circuit documentation for more info.

In [5]:
from qiskit.providers.ibmq import RunnerResult

runtime_params = {
    "circuits": qasm3_reset,
    "shots": 1000,
}

options = {
    "backend_name": runtime_backend.name,
}

qasm3_reset_job = runtime_backend.service.run(
    program_id="qasm3-runner",
    options=options,
    inputs=runtime_params,
    result_decoder=RunnerResult,
)

print(f"Runtime job id: {qasm3_reset_job.job_id}")

Runtime job id: cdjsbiian60ka169ibjg


<ipython-input-5-6204658b2874>:12: DeprecationWarning: Note that the 'job_id' and 'backend' attributes of a runtime job have been deprecated as of qiskit-ibm-runtime 0.7 and will be removed no sooner than 3 months after the release date. Please use the job_id() and backend() methods instead.
  qasm3_reset_job = runtime_backend.service.run(


In [6]:
print(f"The outcome counts from our reset experiment are {qasm3_reset_job.result().get_counts()}")

The outcome counts from our reset experiment are {'0': 23, '1': 940, '10': 13, '11': 24}


## Behind the scenes

![Qiskit Runtime](https://iqx-docs.quantum-computing.ibm.com/_images/Qiskit_Runtime_architecture1.png)

## Currently more capabilities are available through OpenQASM 3 than through Qiskit

Next we see a demo of using extended classical compute capabilities to interplay classical and quantum resources
in real-time on the hardware. We hope that these capabilities will enable a new phase of hardware research.

Note: Classical registers are concatenated together in the results.

In [7]:
classical_compute_qasm3 = """
OPENQASM 3.0;

bit a;
bit b;
bit c;

reset $0; reset $1; reset $2;
barrier $0, $1, $2;
reset $0; reset $1; reset $2;
barrier $0, $1, $2;
reset $0; reset $1; reset $2;
barrier $0, $1, $2;

x $0;
x $2;

a = measure $0; // expected "1"
b = measure $1; // expected "0"
c = measure $2; // expected "1"

bit[3] d = "100";

// Purely classical compute is efficient
if (((a | b) & c) == 1) {
    // Path will nearly always execute 
    d[0] = 1;
} else {
    // Path will rarely execute outside of SPAM errors
    d[0] = 0;
}

bit[3] e = "101";
// Conditionally execute based on classical bit array comparison
// Should always execute
// As it involves a qubit it has a higher latency
if (d == e) {
    x $1;
}

bit final;
final = measure $1; // expected "1"
// Final expected result is '1000000101'
"""

In [8]:
from qiskit.providers.ibmq import RunnerResult

runtime_params = {
    "circuits": classical_compute_qasm3,
    "shots": 1000,
}

options = {
    "backend_name": runtime_backend.name,
}

classical_compute_job = runtime_backend.service.run(
    program_id="qasm3-runner",
    options=options,
    inputs=runtime_params,
    result_decoder=RunnerResult,
)

print(f"Runtime job id: {classical_compute_job.job_id}")

Runtime job id: cdjscdvtlcfkm5erofq0


In [9]:
print(f"The outcome counts from our classical computer experiment {classical_compute_job.result().get_counts()}")

The outcome counts from our classical computer experiment {'1': 10, '1000000100': 1, '1000000101': 936, '1000000111': 12, '100': 25, '101': 12, '111': 4}


## We can also run arbitrary classical code on the controller
We have the capability to run classical code on the controller written in a purely classical language. The hope is that this will allow us and are users to experiment with efficient hardware decoders, real-time variational techniques
and other experimental techniques such as efficient PEC.

*Note*: This is an early feature and is currently in development.


For example, consider the Rust function below which efficiently evaluates the parity of an integer

```rust
#[no_mangle]
pub extern "C" fn parity(mut x: i64) -> i32 {
  x ^= x >> 32;
  x ^= x >> 16;
  x ^= x >> 8;
  x ^= x >> 4;
  x ^= x >> 2;
  x ^= x >> 1;
  return (x as i32) & 1;
}
```

Or this `decoder` function which maps an input state to its bit-flipped inverse.

```rust
#[no_mangle]
pub extern "C" fn decoder(x: i32) -> i32 {
  match x {
    0 => 7,
    1 => 6,
    2 => 5,
    3 => 4,
    4 => 3,
    5 => 2,
    6 => 1,
    7 => 0,
    _ => 0,
  }
}
```

We are able to interplay these classical computations with our quantum dynaimcs through the OpenQASM3 `extern` subroutine declaration as we show below for the `decoder` function above.

The example below shows how to declare an `extern` function that must be made available
by the backend, casting a bit array of measurement results to an integer and using the
result of this to invoke the `decoder` function and then using this result in a hardware
efficient `switch` statement.

In [10]:
parity_check_qasm3 = """
OPENQASM 3;

// Extern definition
extern decoder(int[32]) -> int[32];

// Initialize our qubits
reset $0; reset $1; reset $2; reset $3; reset $4; reset $5;
barrier $0, $1, $2, $3, $4, $5;

// Prepare the input state |101>
x $0; id $1; x $2;
bit[3] prepare;
// Measure this input state
prepare[0] = measure $0;
prepare[1] = measure $1;
prepare[2] = measure $2;
// The output should be "101"

// Cast the bit array to an integer
int[32] decoder_input;
decoder_input  = prepare;

// Invoke the decoder function and assign the result
int[32] decoded;
decoded = decoder(decoder_input);
// The output should be "010"

// Perform a hardware efficient
// switch. Requiring only one
// communication event in the hardware
// for all possible outcomes.
switch (decoded) {
   case 0: {

   }
   break;
   case 1: {
      x $3;
   }
   break;
   // Should be invoked
   case 2: {
      x $4;
   }
   break;
   case 3: {
      x $3;
      x $4;
   }
   break;
   case 4: {
      x $5;
   }
   break;
   case 5: {
      x $3;
      x $5;
   }
   break;
   case 6: {
      x $4;
      x $5;
   } 
   break;
   case 7: {
      x $3;
      x $4;
      x $5;
   }
   break;
   default: {

   }
   break;
}

bit[3] mapped_parity;
mapped_parity[0] = measure $3;
mapped_parity[1] = measure $4;
mapped_parity[2] = measure $5;
// The output should be "010"
"""

In [11]:
from qiskit.providers.ibmq import RunnerResult

runtime_params = {
    "circuits": parity_check_qasm3,
    "shots": 1000,
}

options = {
    "backend_name": runtime_backend.name,
}

parity_check_job = runtime_backend.service.run(
    program_id="qasm3-runner",
    options=options,
    inputs=runtime_params,
    result_decoder=RunnerResult,
)
print(f"Runtime job id: {parity_check_job.job_id}")

Runtime job id: cdjscqi2h6kvivd6cb60


In [13]:
print(f'The correct outcome counts of "010" from our decoder experiment were obtained on {100*parity_check_job.result().get_counts()["10101"]/1000}% of shots')

The correct outcome counts of "010" from our decoder experiment were obtained on 93.8% of shots


Moving forward into 2022 and 2023 we will be working with users to explore the hardware's unique capabilities and enable novel experiments through our internal and client-facing research teams.